In [4]:
import json
import os

In [5]:
dataset_path = '/home/users/nus/idmwyk/scratch/exp/data/videos/msrvtt/'

In [6]:
test_file = os.path.join(dataset_path,'test_videodatainfo.json')

In [7]:
with open(test_file,'r') as f:
    test_info = json.load(f)

In [21]:
annotations_gt = {}
annotations_gt['images'] = test_info['videos']

for sentence in test_info['sentences']:
    sentence['image_id'] = int(sentence['video_id'].replace('video',''))
    sentence['id'] = sentence['video_id']
annotations_gt['annotations'] =test_info['sentences']

In [50]:
import cv2
import numpy as np

def sample_images_from_video(video_path, num_samples=10):
    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Get the total number of frames in the video
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_rate = cap.get(cv2.CAP_PROP_FPS)

    # Calculate total duration in seconds
    total_duration = total_frames / frame_rate
    print(total_duration)
    # Check if the video opened successfully
    if not cap.isOpened():
        print("Error opening video file.")
        return []

    # Calculate the interval for sampling
    interval = total_frames // num_samples

    # Initialize a list to store the sampled images
    sampled_images = []

    for i in range(num_samples):
        # Set the frame position
        frame_id = i * interval
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_id)

        # Read the frame
        ret, frame = cap.read()

        # If frame reading was successful, save the frame
        if ret:
            sampled_images.append(frame)
        else:
            print(f"Error reading frame at position {frame_id}")

    # Release the video capture object
    cap.release()

    return sampled_images, total_frames


In [13]:
from pycocotools.coco import COCO
from pycocoevalcap.eval import COCOEvalCap

In [14]:
import json
from json import encoder
encoder.FLOAT_REPR = lambda o: format(o, '.3f')
import sys

In [22]:
annotation_file = os.path.join(dataset_path,'annotation_file')
with open(annotation_file,'w') as f:
    json.dump(annotations_gt,f)
coco = COCO(annotation_file)

loading annotations into memory...
Done (t=0.06s)
creating index...
index created!


In [23]:
video_ids = coco.getImgIds()

In [24]:
len(video_ids)

2990

In [25]:
results = []
for video in video_ids:
    results.append({'image_id':video,'caption': 'a video'})

In [27]:
coco_result = coco.loadRes(results) 

coco_eval = COCOEvalCap(coco, coco_result)
coco_eval.evaluate()
# print output evaluation scores
for metric, score in coco_eval.eval.items():
    print(f"{metric}: {score:.3f}")

Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...


PTBTokenizer tokenized 617049 tokens at 2968232.24 tokens per second.
PTBTokenizer tokenized 8969 tokens at 180713.53 tokens per second.


setting up scorers...
computing Bleu score...
{'testlen': 5980, 'reflen': 16015, 'guess': [5980, 2990, 0, 0], 'correct': [4514, 869, 0, 0]}
ratio: 0.3733999375585155
Bleu_1: 0.141
Bleu_2: 0.087
Bleu_3: 0.001
Bleu_4: 0.000
computing METEOR score...
METEOR: 0.080
computing Rouge score...
ROUGE_L: 0.302
computing CIDEr score...
CIDEr: 0.022
computing SPICE score...


Parsing reference captions
Initiating Stanford parsing pipeline
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator tokenize
[main] INFO edu.stanford.nlp.pipeline.TokenizerAnnotator - TokenizerAnnotator: No tokenizer type provided. Defaulting to PTBTokenizer.
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ssplit
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator parse
[main] INFO edu.stanford.nlp.parser.common.ParserGrammar - Loading parser from serialized file edu/stanford/nlp/models/lexparser/englishPCFG.ser.gz ... 
done [0.4 sec].
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator lemma
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ner
Loading classifier from edu/stanford/nlp/models/ner/english.all.3class.distsim.crf.ser.gz ... done [0.8 sec].
Loading classifier from edu/stanford/nlp/models/ner/english.muc.7class.distsim.crf.ser.gz ... done [0.4 sec].
Loading classif

SPICE evaluation took: 2.058 min
SPICE: 0.016
Bleu_1: 0.141
Bleu_2: 0.087
Bleu_3: 0.001
Bleu_4: 0.000
METEOR: 0.080
ROUGE_L: 0.302
CIDEr: 0.022
SPICE: 0.016
